In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from IPython.display import display,HTML

# NLP 深入研究：RNN

在<<chapter_intro>>中，我们看到深度学习可以用于自然语言数据集并取得出色的结果。我们的例子依赖于使用预训练的语言模型并对其进行微调以对评论进行分类。这个例子突出了自然语言处理（NLP）中的迁移学习与计算机视觉中的迁移学习之间的差异：一般来说，在NLP中，预训练模型是在不同任务上训练的。

我们所说的语言模型是一种已经训练好的模型，它能够猜测文本中下一个单词是什么（在阅读了前面的单词之后）。这种任务被称为*自监督学习*：我们不需要给我们的模型提供标签，只需要给它大量的文本。它有一个从数据中自动获取标签的过程，这个任务并不简单：要正确猜测句子中的下一个单词，模型必须发展出对英语（或其他语言）的理解。自监督学习也可以在其他领域使用；例如，参见["自监督学习与计算机视觉"](https://www.fast.ai/2020/01/13/self_supervised/) 以了解视觉应用的介绍。自监督学习通常不用于直接训练的模型，而是用于预训练用于迁移学习的模型。

> 术语：自监督学习：使用嵌入在独立变量中的标签来训练模型，而不是需要外部标签。例如，训练一个模型来预测文本中的下一个单词。

我们在<<chapter_intro>>中用于对IMDb评论进行分类的语言模型是在Wikipedia上预训练的。通过直接对这个语言模型进行微调以适应电影评论分类器，我们取得了很好的结果，但是通过一个额外的步骤，我们可以做得更好。Wikipedia上的英语与IMDb上的英语略有不同，因此，我们可以先对预训练的语言模型进行微调以适应IMDb语料库，然后再将其作为我们分类器的基础。

即使我们的语言模型知道任务中使用的语言的基础知识（例如，我们的预训练模型是英文），适应我们目标语料库的风格也是有帮助的。它可能是更非正式的语言，或者更技术性，有新词需要学习，或者句子构造方式不同。在IMDb数据集的情况下，会有很多电影导演和演员的名字，通常语言风格比Wikipedia上看到的要非正式。

我们已经看到，使用fastai，我们可以下载一个预训练的英文语言模型，并用它来为自然语言处理（NLP）分类任务获得最先进的结果。（我们预计很快就会有更多的预训练模型提供更多语言——实际上，到你阅读这本书的时候可能已经有了。）那么，为什么我们要学习如何详细训练一个语言模型呢？

当然，一个原因是了解你正在使用的模型的基础是有帮助的。但还有一个非常实际的原因，那就是在微调分类模型之前对（基于序列的）语言模型进行微调，你会得到更好的结果。例如，对于IMDb情感分析任务，数据集包括50,000篇额外的电影评论，这些评论没有任何正面或负面的标签。由于训练集中有25,000篇带标签的评论，验证集中也有25,000篇，总共有100,000篇电影评论。我们可以使用所有这些评论来微调预训练的语言模型，这个模型只在Wikipedia文章上训练过；这将产生一个特别擅长预测电影评论下一个字的语言模型。

这被称为通用语言模型微调（ULMFit）方法。[论文](https://arxiv.org/abs/1801.06146) 显示，在将语言模型转移到分类任务之前，对语言模型进行额外的微调阶段，可以显著提高预测的准确性。使用这种方法，我们在NLP中有转移学习的三个阶段，如<<ulmfit_process>>所总结。

<img alt="Diagram of the ULMFiT process" width="700" caption="The ULMFiT process" id="ulmfit_process" src="images/att_00027.png">

现在我们将探讨如何将神经网络应用于这个语言建模问题，使用前两章介绍的概念。但在继续阅读之前，请暂停一下，思考一下你会如何处理这个问题。

## 文本预处理

我们如何使用到目前为止学到的知识来构建一个语言模型并不明显。句子的长度可以不同，文档可以非常长。那么，我们如何使用神经网络预测一个句子的下一个单词呢？让我们找出答案！

我们已经看到了分类变量如何作为神经网络的独立变量。我们对单个分类变量的方法是：

1. 列出该分类变量所有可能的级别（我们将这个列表称为*词汇表*）。
2. 用词汇表中的索引替换每个级别。
3. 为这个词汇表创建一个嵌入矩阵，其中包含每个级别（即词汇表中的每个项目）的一行。
4. 使用这个嵌入矩阵作为神经网络的第一层。（专用的嵌入矩阵可以接受在第二步中创建的原始词汇表索引作为输入；这相当于但比接受代表索引的独热编码向量的矩阵更快、更高效。）

我们几乎可以用同样的方法处理文本！新的想法是序列。首先，我们将数据集中的所有文档连接成一个很长的字符串，然后将其分割成单词，得到一个非常长的单词列表（或“标记”）。我们的独立变量将是单词序列，从我们非常长的列表中的第一个单词开始，到倒数第二个单词结束，而我们的因变量将是单词序列，从第二个单词开始，到最后一个单词结束。

我们的词汇表将包含已经在我们预训练模型的词汇表中的常见单词和特定于我们语料库的新单词（例如电影术语或演员名字）。我们的嵌入矩阵将相应地构建：对于预训练模型词汇表中的单词，我们将取预训练模型嵌入矩阵中对应的行；但对于新单词，我们没有任何东西，所以我们只会用一个随机向量初始化对应的行。

创建语言模型所需的每个步骤都与自然语言处理领域的术语相关，并且fastai和PyTorch提供了相应的类来帮助实现。这些步骤包括：

- **分词（Tokenization）**：将文本转换成单词列表（或字符、子字符串，取决于模型的粒度）。
- **数值化（Numericalization）**：列出所有出现的单词（即词汇表），并通过在词汇表中查找其索引将每个单词转换为数字。
- **语言模型数据加载器创建**：fastai提供了一个`LMDataLoader`类，它自动处理创建一个因变量，该因变量与独立变量相差一个标记。它还处理一些重要细节，例如如何以一种保持依赖和独立变量结构所必需的方式对训练数据进行洗牌。
- **语言模型创建**：我们需要一种特殊的模型，它能够处理大小可能任意的输入列表。实现这一点有多种方法；在本章中，我们将使用*循环神经网络*（RNN）。我们将在<<chapter_nlp_dive>>中详细介绍这些RNN的细节，但目前你可以将其视为另一种深度神经网络。

让我们详细看看每个步骤是如何工作的。

### 分词

当我们说“将文本转换成单词列表”时，我们省略了很多细节。例如，我们如何处理标点符号？像“don't”这样的词怎么办？它是一个词还是两个词？对于长医学或化学单词，我们应该将它们拆分成它们各自的含义部分吗？连字符单词怎么办？对于像德语和波兰语这样的语言，我们可以从许多部分创建非常长的单词，我们该怎么办？对于像日语和汉语这样的语言，它们根本不使用基础，也没有明确定义的“词”概念怎么办？

因为这些问题没有唯一正确的答案，所以分词也没有一种统一的方法。主要有三种主要方法：

- **基于单词**：根据空格分割句子，同时应用特定于语言的规则尝试在没有空格的情况下分离含义的部分（例如将“don't”变成“do n't”）。通常，标点符号也被分割成单独的标记。
- **基于子词**：基于最常见的子字符串将单词分割成更小的部分。例如，“occasion”可能被分词为“o c ca sion”。
- **基于字符**：将句子分割成其单个字符。

我们将在这里看基于单词和基于子词的分词，我们将把基于字符的分词留给你在本章末的问卷中实现。

> 术语：标记（token）：分词过程中创建的列表中的一个元素。它可以是一个单词、单词的一部分（子词），或者是一个单独的字符。

### 使用fastai进行单词分词

fastai没有提供自己的分词器，而是为外部库中的一系列分词器提供了一致的接口。分词是一个活跃的研究领域，新的和改进的分词器不断出现，因此fastai使用的默认设置也会随之改变。然而，API和选项应该不会有太大变化，因为fastai试图在底层技术变化的同时保持一致的API。

让我们用我们在<<chapter_intro>>中使用的IMDb数据集来试试：

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

为了尝试分词器，我们需要获取文本文件。就像我们已经多次使用的`get_image_files`一样，它获取路径中的所有图像文件，`get_text_files`获取路径中的所有文本文件。我们还可以可选地传递`folders`来限制搜索到特定的子文件夹列表：

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

这是我们将要分词的一篇评论（为了节省空间，我们只打印它的开头部分）：

In [ ]:
txt = files[0].open().read(); txt[:75]

'This movie, which I just discovered at the video store, has apparently sit '

在我们写这本书的时候，fastai的默认英文单词分词器使用的是一个叫做*spaCy*的库。它有一个复杂的规则引擎，对URL、个别特殊英文单词等有特殊规则。然而，我们不会直接使用`SpacyTokenizer`，而是会使用`WordTokenizer`，因为那将始终指向fastai当前的默认单词分词器（根据你阅读此内容的时间，它可能不一定是spaCy）。

让我们试试。我们将使用fastai的`coll_repr(collection, n)`函数来显示结果。这个函数显示*`collection`*的前*`n`*个项目，以及完整的大小——这是`L`默认使用的。请注意，fastai的分词器接受一组文档进行分词，所以我们需要将`txt`包装在一个列表中：

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#201) ['This','movie',',','which','I','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','It',"'s",'easy','to','see'...]


正如你所看到的，spaCy主要只是将单词和标点符号分开。但它在这里还做了其他事情：它将"it's"分成了"it"和"'s"。这是有道理的；这些实际上是分开的单词。当你考虑到所有必须处理的小细节时，分词实际上是一个相当微妙的任务。幸运的是，spaCy为我们处理得相当好——例如，在这里我们看到"."在结束句子时被分开，但在缩写词或数字中则没有分开：

In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

fastai通过`Tokenizer`类在分词过程中增加了一些额外的功能：

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#228) ['xxbos','xxmaj','this','movie',',','which','i','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','xxmaj','it',"'s",'easy'...]


注意，现在有一些标记以"xx"字符开头，这在英语中不是常见的单词前缀。这些是*特殊标记*。

例如，列表中的第一项`xxbos`是一个特殊标记，表示新文本的开始（"BOS"是NLP领域的标准缩写，意思是“流的开始”）。通过识别这个开始标记，模型将能够学会“忘记”之前所说的内容，专注于即将到来的单词。

这些特殊标记并不直接来自spaCy。它们之所以存在，是因为fastai在处理文本时默认应用了一系列规则。这些规则旨在帮助模型更容易识别句子的重要部分。在某种意义上，我们正在将原始的英语语言序列翻译成一种简化的标记语言——一种为模型学习而设计的简化语言。

例如，规则将连续的四个感叹号替换为一个特殊的*重复字符*标记，后面跟着数字四，然后是一个单独的感叹号。这样，模型的嵌入矩阵可以编码关于重复标点等一般概念的信息，而不需要为每种标点符号的每种重复次数单独设置标记。同样，大写单词将被替换为一个特殊的大写标记，后面跟着单词的小写版本。这样，嵌入矩阵只需要单词的小写版本，节省计算和内存资源，但仍然可以学习大写的概念。

以下是你将看到的一些主要特殊标记：

- `xxbos`:: 表示文本的开始（这里是一个评论）
- `xxmaj`:: 表示下一个单词以大写字母开头（因为我们已经将所有内容转换为小写）
- `xxunk`:: 表示单词未知

要查看使用的规则，你可以检查默认规则：

In [ ]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

一如既往，你可以通过在笔记本中输入以下代码查看它们的源代码：

```
??replace_rep
```

以下是每个函数的简要总结：

- `fix_html`:: 将特殊HTML字符替换为可读版本（IMDb评论中有很多这样的字符）
- `replace_rep`:: 将任何重复三次或更多次的字符替换为重复的特殊标记（`xxrep`），重复的次数，然后是字符本身
- `replace_wrep`:: 将任何重复三次或更多次的单词替换为单词重复的特殊标记（`xxwrep`），重复的次数，然后是单词本身
- `spec_add_spaces`:: 在/和#周围添加空格
- `rm_useless_spaces`:: 删除所有空格字符的重复
- `replace_all_caps`:: 将全部大写的单词转换为小写，并在其前面添加一个特殊标记表示全部大写（`xxup`）
- `replace_maj`:: 将首字母大写的单词转换为小写，并在其前面添加一个特殊标记表示首字母大写（`xxmaj`）
- `lowercase`:: 将所有文本转换为小写，并在开头（`xxbos`）和/或结尾（`xxeos`）添加特殊标记

让我们看看其中一些在实际操作中的样子：

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index'...]"

现在让我们看看基于子词的分词是如何工作的。

### 基于子词的分词

除了上一节中看到的*单词分词*方法外，另一种流行的分词方法是*基于子词的分词*。单词分词依赖于一个假设，即空格在句子中提供了意义成分的有用分隔。然而，这个假设并不总是适用。例如，考虑这个句子：我的名字是郝杰瑞（中文中的“My name is Jeremy Howard”）。这对于单词分词器来说不会很好用，因为它里面没有空格！像中文和日文这样的语言不使用空格，实际上它们甚至没有一个明确定义的“词”概念。还有一些语言，如土耳其语和匈牙利语，可以在没有空格的情况下将许多子词组合在一起，创造出包含大量独立信息的非常长的单词。

为了处理这些情况，通常最好使用基于子词的分词。这分为两个步骤：

1. 分析文档语料库，找到最常见的字母组合。这些成为词汇表。
2. 使用这个*子词单位*的词汇表对语料库进行分词。

让我们来看一个例子。对于我们的语料库，我们将使用前2,000篇电影评论：

In [ ]:
txts = L(o.open().read() for o in files[:2000])

我们实例化我们的分词器，传入我们想要创建的词汇表大小，然后我们需要“训练”它。也就是说，我们需要让它读取我们的文档并找到常见的字符序列来创建词汇表。这是通过`setup`完成的。正如我们很快就会看到的，`setup`是一个特殊的fastai方法，它在我们通常的数据处理流程中会自动调用。然而，由于我们目前手动完成所有事情，我们必须自己调用它。以下是一个执行这些步骤的函数，并展示了一个示例输出：

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

让我们试试：

In [ ]:
subword(1000)

'▁This ▁movie , ▁which ▁I ▁just ▁dis c over ed ▁at ▁the ▁video ▁st or e , ▁has ▁a p par ent ly ▁s it ▁around ▁for ▁a ▁couple ▁of ▁years ▁without ▁a ▁dis t ri but or . ▁It'

在使用fastai的基于子词的分词器时，特殊字符` `代表原始文本中的空格字符。

如果我们使用较小的词汇表，那么每个标记将代表更少的字符，表示一个句子需要更多的标记：

In [ ]:
subword(200)

'▁ T h i s ▁movie , ▁w h i ch ▁I ▁ j us t ▁ d i s c o ver ed ▁a t ▁the ▁ v id e o ▁ st or e , ▁h a s'

另一方面，如果我们使用较大的词汇表，那么大多数常见的英文单词最终会出现在词汇表中，我们不需要那么多标记来表示一个句子：

In [ ]:
subword(10000)

"▁This ▁movie , ▁which ▁I ▁just ▁discover ed ▁at ▁the ▁video ▁store , ▁has ▁apparently ▁sit ▁around ▁for ▁a ▁couple ▁of ▁years ▁without ▁a ▁distributor . ▁It ' s ▁easy ▁to ▁see ▁why . ▁The ▁story ▁of ▁two ▁friends ▁living"

选择一个基于子词的词汇表大小是一种折中：较大的词汇表意味着每个句子的标记更少，这意味着训练更快，内存需求更少，模型需要记住的状态也更少；但缺点是，它意味着更大的嵌入矩阵，这需要更多的数据来学习。

总的来说，基于子词的分词提供了一种在字符分词（即使用小的子词词汇表）和单词分词（即使用大的子词词汇表）之间轻松调整的方法，并且能够处理每一种人类语言，而不需要开发特定于语言的算法。它甚至可以处理其他“语言”，如基因序列或MIDI音乐符号！因此，在过去的一年里，它的受欢迎程度飙升，并且很可能成为最常见的分词方法（到你阅读这篇文章的时候可能已经是了！）。

一旦我们的文本被分割成标记，我们就需要将它们转换为数字。接下来我们将讨论这个问题。

### 使用fastai进行数值化

*数值化*是将标记映射到整数的过程。这些步骤基本上与创建`Category`变量所需的步骤相同，例如MNIST中数字的因变量：

1. 列出该分类变量所有可能的级别（词汇表）。
2. 用词汇表中的索引替换每个级别。

让我们看看之前看到的单词分词文本的实际操作：

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#228) ['xxbos','xxmaj','this','movie',',','which','i','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','xxmaj','it',"'s",'easy'...]


就像使用`SubwordTokenizer`一样，我们需要在`Numericalize`上调用`setup`；这是我们创建词汇表的方式。这意味着我们首先需要分词的语料库。由于分词需要一段时间，fastai会并行执行；但为了这个手动演练，我们将使用一个小的子集：

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#228) ['xxbos','xxmaj','this','movie',',','which','i','just','discovered','at'...]

我们可以将其传递给`setup`来创建我们的词汇表：

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2000) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','in','i','it'...]"

我们的特殊规则标记首先出现，然后每个单词按频率顺序出现一次。`Numericalize`的默认设置是`min_freq=3,max_vocab=60000`。`max_vocab=60000`导致fastai将除了最常见的60,000个单词之外的所有单词替换为特殊的*未知词*标记`xxunk`。这有助于避免嵌入矩阵过大，因为这会减慢训练速度并占用太多内存，也可能意味着没有足够的数据来训练罕见单词的有用表示。然而，最后一个问题最好通过设置`min_freq`来处理；默认的`min_freq=3`意味着任何出现少于三次的单词都将被替换为`xxunk`。

fastai还可以使用你提供的词汇表对你的数据集进行数值化，通过将单词列表作为`vocab`参数传递。

一旦我们创建了`Numericalize`对象，我们就可以像使用函数一样使用它：

In [ ]:
nums = num(toks)[:20]; nums

tensor([  2,   8,  21,  28,  11,  90,  18,  59,   0,  45,   9, 351, 499,  11,  72, 533, 584, 146,  29,  12])

这一次，我们的标记已经被转换为模型可以接收的整数张量。我们可以检查它们是否映射回原始文本：

In [ ]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj this movie , which i just xxunk at the video store , has apparently sit around for a'

现在我们有了数字，我们需要将它们分批放入我们的模型中。

### 将我们的文本分批输入语言模型

在处理图像时，我们需要先将它们全部调整到相同的高度和宽度，然后将它们分组到一个小批量中，以便它们可以有效地堆叠成一个单一的张量。在这里情况会有所不同，因为我们不能简单地调整文本到期望的长度。此外，我们希望我们的语言模型能够按顺序阅读文本，这样它才能有效地预测下一个单词是什么。这意味着每个新的批次应该精确地从上一个批次结束的地方开始。

假设我们有以下文本：

> : 在这一章中，我们将回顾第一章中学习的对电影评论进行分类的例子，并深入探讨。首先，我们将看看将文本转换为数字所需的处理步骤以及如何定制它。通过这样做，我们将有另一个数据块API中使用的PreProcessor的例子。
然后，我们将研究如何构建一个语言模型并训练一段时间。

令牌化过程将添加特殊令牌并处理标点符号，返回以下文本：

> : xxbos xxmaj 在这一章中，我们将回顾第一章中学习的对电影评论进行分类的例子，并深入探讨。xxmaj 首先，我们将看看将文本转换为数字所需的处理步骤以及如何定制它。xxmaj 通过这样做，我们将有另一个数据块xxup api中使用的预处理器的例子。
> xxmaj 然后，我们将研究如何构建一个语言模型并训练一段时间。

现在我们有90个令牌，由空格分隔。假设我们想要一个大小为6的批次。我们需要将这段文本分成6个连续的部分，每部分长度为15：

In [ ]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


在一个理想的世界里，我们可以将这一个批次直接交给我们的模型。但这种方法并不具有可扩展性，因为在这个玩具示例之外，包含所有文本的单个批次很可能无法适应我们的GPU内存（这里我们有90个令牌，但所有的IMDb评论加起来有几千万个）。

因此，我们需要将这个数组更精细地划分为固定序列长度的子数组。保持这些子数组内部以及它们之间的顺序是很重要的，因为我们将使用一个保持状态的模型，以便在预测接下来的内容时记住它之前阅读的内容。

回到我们之前的例子，有6个长度为15的批次，如果我们选择一个长度为5的序列，那将意味着我们首先输入以下数组：

In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


Then this one:

In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


最后:

In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


回到我们的电影评论数据集，第一步是通过连接它们将各个文本转换成一个流。与图像一样，最好随机化输入的顺序，所以在每个时代的开始，我们会打乱条目的顺序来创建一个新的流（我们打乱文档的顺序，而不是它们内部单词的顺序，否则文本就不再有意义了！）。

然后我们将这个流切割成一定数量的批次（这是我们的*批次大小*）。例如，如果流有50,000个令牌，我们设置批次大小为10，这将给我们10个包含5,000个令牌的迷你流。重要的是我们要保留令牌的顺序（所以第一个迷你流是从1到5,000，然后是从5,001到10,000...），因为我们希望模型读取连续的文本行（如前一个例子所示）。在预处理期间，每个开头都会添加一个`xxbos`令牌，这样模型就知道在读取流时一个新的条目开始了。

所以总结一下，每个时代我们都会打乱我们的文档集合并将它们连接成一个令牌流。然后我们将这个流切割成固定大小的连续迷你流的批次。我们的模型将按顺序读取这些迷你流，并且由于内部状态的存在，无论我们选择的序列长度是多少，它都会产生相同的激活。

所有这些都是在创建`LMDataLoader`时由fastai库在后台完成的。我们首先将我们的`Numericalize`对象应用到令牌化的文本上：

In [ ]:
nums200 = toks200.map(num)

然后，我们将处理后的数据传递给LMDataLoader：

In [ ]:
dl = LMDataLoader(nums200)

为了确认这给出了预期的结果，我们可以通过获取第一个批次来进行检查：

In [ ]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

然后，我们可以查看独立变量（通常是输入序列）的第一行，它应该是第一条文本的开始：

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj this movie , which i just xxunk at the video store , has apparently sit around for a'

因变量是相同的内容，但偏移了一个令牌：

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj this movie , which i just xxunk at the video store , has apparently sit around for a couple'

这结束了我们需要对我们的数据应用的所有预处理步骤。现在我们准备好训练我们的文本分类器了。

## 训练文本分类器

正如我们在本章开头看到的，使用迁移学习训练一个最先进的文本分类器有两个步骤：首先，我们需要将我们在维基百科上预训练的语言模型微调到IMDb评论的语料库上，然后我们可以使用该模型来训练一个分类器。

和往常一样，让我们从组装我们的数据开始。

### 使用DataBlock的语言模型

当`TextBlock`传递给`DataBlock`时，fastai会自动处理令牌化和数值化。可以传递给`Tokenize`和`Numericalize`的所有参数也可以传递给`TextBlock`。在下一章中，我们将讨论分别运行这些步骤的最简单方法，以便于调试——但你总是可以通过像前几节所示的那样手动在数据子集上运行它们来进行调试。不要忘记`DataBlock`的便捷`summary`方法，它对于调试数据问题非常有用。

以下是我们如何使用`TextBlock`来创建语言模型，使用fastai的默认设置：

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

在`DataBlock`中我们使用的一个与之前不同的点是我们不是直接使用类（即`TextBlock(...)`），而是调用一个*类方法*。类方法是一个Python方法，正如其名，它属于一个*类*而不是一个*对象*。（如果你不熟悉类方法，请务必在线搜索更多信息，因为它们在许多Python库和应用程序中都很常见；我们在书中之前也使用过它们几次，但没有特别指出。）`TextBlock`之所以特殊，是因为设置数值器的词汇表可能需要很长时间（我们必须读取并令牌化每个文档以获取词汇表）。为了尽可能高效，它执行了一些优化：

- 它将令牌化的文档保存在一个临时文件夹中，这样就不用多次令牌化它们
- 它并行运行多个令牌化进程，以利用你的计算机的CPU

我们需要告诉`TextBlock`如何访问文本，以便它可以进行这个初始的预处理——这就是`from_folder`的作用。

然后`show_batch`就像通常一样工作：

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj it 's awesome ! xxmaj in xxmaj story xxmaj mode , your going from punk to pro . xxmaj you have to complete goals that involve skating , driving , and walking . xxmaj you create your own skater and give it a name , and you can make it look stupid or realistic . xxmaj you are with your friend xxmaj eric throughout the game until he betrays you and gets you kicked off of the skateboard","xxmaj it 's awesome ! xxmaj in xxmaj story xxmaj mode , your going from punk to pro . xxmaj you have to complete goals that involve skating , driving , and walking . xxmaj you create your own skater and give it a name , and you can make it look stupid or realistic . xxmaj you are with your friend xxmaj eric throughout the game until he betrays you and gets you kicked off of the skateboard xxunk"
1,"what xxmaj i 've read , xxmaj death xxmaj bed is based on an actual dream , xxmaj george xxmaj barry , the director , successfully transferred dream to film , only a genius could accomplish such a task . \n\n xxmaj old mansions make for good quality horror , as do portraits , not sure what to make of the killer bed with its killer yellow liquid , quite a bizarre dream , indeed . xxmaj also , this","xxmaj i 've read , xxmaj death xxmaj bed is based on an actual dream , xxmaj george xxmaj barry , the director , successfully transferred dream to film , only a genius could accomplish such a task . \n\n xxmaj old mansions make for good quality horror , as do portraits , not sure what to make of the killer bed with its killer yellow liquid , quite a bizarre dream , indeed . xxmaj also , this is"


现在我们的数据已经准备好了，我们可以对预训练的语言模型进行微调。

### 微调语言模型

为了将整数单词索引转换成我们神经网络可以使用的激活值，我们将使用嵌入层，就像我们在协同过滤和表格建模中所做的一样。然后我们将这些嵌入输入到一个名为*循环神经网络*（RNN）的结构中，使用一种称为*AWD-LSTM*的架构（我们将在<<chapter_nlp_dive>>中展示如何从头开始编写这样的模型）。正如我们之前讨论的，预训练模型中的嵌入会与为不在预训练词汇表中的单词添加的随机嵌入合并。这在`language_model_learner`内部会自动处理：

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

默认使用的损失函数是交叉熵损失，因为我们本质上有一个分类问题（不同的类别是我们词汇表中的单词）。这里使用的*困惑度*（perplexity）指标经常在自然语言处理（NLP）中用于语言模型：它是损失的指数（即，`torch.exp(cross_entropy)`）。我们还包括准确率指标，以了解我们的模型在尝试预测下一个单词时正确了多少次，因为交叉熵（正如我们所看到的）既难以解释，又更多地告诉我们模型的置信度而不是其准确性。

让我们回顾一下本章开头的流程图。第一个箭头已经为我们完成，并作为预训练模型在fastai中提供，我们刚刚为第二阶段构建了`DataLoaders`和`Learner`。现在我们准备好微调我们的语言模型了！

<img alt="Diagram of the ULMFiT process" width="450" src="images/att_00027.png">

训练每个时代需要相当长的时间，所以我们将在训练过程中保存中间模型的结果。由于`fine_tune`不会为我们做这件事，我们将使用`fit_one_cycle`。就像`vision_learner`一样，`language_model_learner`在使用预训练模型时（默认情况下）会自动调用`freeze`，所以这将只训练嵌入层（模型中唯一包含随机初始化权重的部分——即，在我们IMDb词汇表中但不在预训练模型词汇表中的单词的嵌入层）：

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.120048,3.912788,0.299565,50.038246,11:39


这个模型训练需要一段时间，所以这是一个很好的机会来讨论保存中间结果。

### 保存和加载模型

你可以这样轻松地保存你的模型状态：

In [ ]:
learn.save('1epoch')

这将在 `learn.path/models/` 中创建一个名为 *1epoch.pth* 的文件。如果你想在以相同的方式创建你的 `Learner` 后在另一台机器上加载你的模型，或者稍后恢复训练，你可以使用以下方式加载这个文件的内容：

In [ ]:
learn = learn.load('1epoch')

一旦初始训练完成，我们可以在解冻后继续微调模型：

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.893486,3.772820,0.317104,43.502548,12:37
1,3.820479,3.717197,0.323790,41.148880,12:30
2,3.735622,3.659760,0.330321,38.851997,12:09
3,3.677086,3.624794,0.333960,37.516987,12:12
4,3.636646,3.601300,0.337017,36.645859,12:05
5,3.553636,3.584241,0.339355,36.026001,12:04
6,3.507634,3.571892,0.341353,35.583862,12:08
7,3.444101,3.565988,0.342194,35.374371,12:08
8,3.398597,3.566283,0.342647,35.384815,12:11
9,3.375563,3.568166,0.342528,35.451500,12:05


完成这些后，我们保存我们模型的所有部分，除了最后一层，这层将激活值转换为我们词汇表中每个令牌被选择的概率。不包括最后一层的模型被称为*编码器*。我们可以使用`save_encoder`来保存它：

In [ ]:
learn.save_encoder('finetuned')

> 术语解释：编码器（Encoder）：不包括特定任务的最终层（或层）的模型。当应用于视觉CNN时，这个术语与“主体”（body）意思相近，但“编码器”在自然语言处理（NLP）和生成模型中更常用。

这完成了文本分类过程的第二阶段：微调语言模型。现在我们可以使用它来使用IMDb情感标签微调一个分类器。

### 文本生成

在我们继续进行分类器的微调之前，让我们快速尝试一些不同的事情：使用我们的模型来生成随机评论。由于它被训练来猜测句子的下一个单词是什么，我们可以使用模型来编写新的评论：

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

i liked this movie because of its story and characters . The story line was very strong , very good for a sci - fi film . The main character , Alucard , was very well developed and brought the whole story
i liked this movie because i like the idea of the premise of the movie , the ( very ) convenient virus ( which , when you have to kill a few people , the " evil " machine has to be used to protect


正如你所看到的，我们添加了一些随机性（我们根据模型返回的概率随机选择一个单词），这样我们就不会得到完全相同的评论两次。我们的模型没有任何关于句子结构或语法规则的编程知识，然而它显然已经学会了很多关于英语句子的知识：我们可以看到它正确地使用了大写字母（*I* 只是变成了 *i*，因为我们的规则要求至少有两个字符才将一个单词视为大写的，所以看到它小写是正常的），并且使用了一致的时态。乍一看，这篇评论大体上是有意义的，只有当你仔细阅读时，你才会注意到有些地方有点不对劲。对于一个在几个小时内训练出来的模型来说，这已经很不错了！

但我们的最终目标不是训练一个生成评论的模型，而是对它们进行分类...所以让我们使用这个模型来做这件事。

### 创建分类器数据加载器

我们现在从语言模型微调转向分类器微调。回顾一下，语言模型预测文档的下一个单词，因此它不需要任何外部标签。然而，分类器预测一些外部标签——在IMDb的情况下，它是文档的情感。

这意味着我们用于NLP分类的`DataBlock`结构看起来非常熟悉。实际上，它几乎与我们处理过的许多图像分类数据集的结构相同：

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

就像图像分类一样，`show_batch`显示了因变量（在本例中为情感）与每个自变量（电影评论文本）：

In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos i rate this movie with 3 skulls , only coz the girls knew how to scream , this could 've been a better movie , if actors were better , the twins were xxup ok , i believed they were evil , but the eldest and youngest brother , they sucked really bad , it seemed like they were reading the scripts instead of acting them … . spoiler : if they 're vampire 's why do they freeze the blood ? vampires ca n't drink frozen blood , the sister in the movie says let 's drink her while she is alive … .but then when they 're moving to another house , they take on a cooler they 're frozen blood . end of spoiler \n\n it was a huge waste of time , and that made me mad coz i read all the reviews of how",neg
1,"xxbos i have read all of the xxmaj love xxmaj come xxmaj softly books . xxmaj knowing full well that movies can not use all aspects of the book , but generally they at least have the main point of the book . i was highly disappointed in this movie . xxmaj the only thing that they have in this movie that is in the book is that xxmaj missy 's father comes to xxunk in the book both parents come ) . xxmaj that is all . xxmaj the story line was so twisted and far fetch and yes , sad , from the book , that i just could n't enjoy it . xxmaj even if i did n't read the book it was too sad . i do know that xxmaj pioneer life was rough , but the whole movie was a downer . xxmaj the rating",neg
2,"xxbos xxmaj this , for lack of a better term , movie is lousy . xxmaj where do i start … … \n\n xxmaj cinemaphotography - xxmaj this was , perhaps , the worst xxmaj i 've seen this year . xxmaj it looked like the camera was being tossed from camera man to camera man . xxmaj maybe they only had one camera . xxmaj it gives you the sensation of being a volleyball . \n\n xxmaj there are a bunch of scenes , haphazardly , thrown in with no continuity at all . xxmaj when they did the ' split screen ' , it was absurd . xxmaj everything was squished flat , it looked ridiculous . \n\n xxmaj the color tones were way off . xxmaj these people need to learn how to balance a camera . xxmaj this ' movie ' is poorly made , and",neg


查看`DataBlock`定义，每部分都与我们之前构建的数据块非常相似，有两个重要的例外：

- `TextBlock.from_folder`不再有`is_lm=True`参数。
- 我们传递了我们为语言模型微调创建的`vocab`。

我们传递语言模型的`vocab`是为了确保我们使用相同的令牌到索引的对应关系。否则，我们在微调的语言模型中学到的嵌入对这个模型来说将没有任何意义，微调步骤也不会有任何用处。

通过传递`is_lm=False`（或者根本不传递`is_lm`，因为它默认为`False`），我们告诉`TextBlock`我们有常规的标记数据，而不是使用下一个令牌作为标签。然而，我们必须处理一个挑战，这与将多个文档组合成一个小批量有关。让我们通过一个例子来看看，尝试创建一个包含前10个文档的小批量。首先我们将它们数值化：

In [ ]:
nums_samp = toks200[:10].map(num)

现在让我们看看这10个电影评论各自有多少个令牌：

In [ ]:
nums_samp.map(len)

(#10) [228,238,121,290,196,194,533,124,581,155]

请记住，PyTorch的`DataLoader`需要将批次中的所有项合并成一个单一的张量，而单一张量的形状是固定的（即，它在每个轴上都有特定的长度，所有项都必须一致）。这听起来应该很熟悉：我们在处理图像时遇到了同样的问题。在那种情况下，我们使用裁剪、填充和/或压缩来使所有输入的大小相同。对于文档来说，裁剪可能不是一个好主意，因为这很可能会移除一些关键信息（话说回来，对于图像也是如此，我们在那里使用裁剪；数据增强在NLP中还没有得到很好的探索，所以也许实际上在NLP中使用裁剪也有机会！）。你不能真正“压缩”一个文档。所以这就剩下填充了！

我们将扩展最短的文本，使它们的大小相同。为此，我们使用一个特殊的填充令牌，模型会忽略它。此外，为了避免内存问题并提高性能，我们将大致相同长度的文本组合在一起（训练集会有一些随机化）。我们通过（对于训练集，大致上）在每个时代之前按长度排序文档来实现这一点。这样做的结果是，合并到单个批次中的文档往往会有相似的长度。我们不会将每个批次都填充到相同的大小，而是使用每个批次中最长文档的大小作为目标大小。（对于图像也可以做类似的事情，这对于不规则大小的矩形图像特别有用，但在撰写本文时，还没有库为此提供很好的支持，也没有论文涵盖这一点。不过，我们计划很快在fastai中添加这个功能，所以请留意本书的网站；一旦我们让它运行良好，我们会添加相关信息。）

排序和填充是由数据块API自动为我们完成的，当使用`TextBlock`并设置`is_lm=False`时。（我们对语言模型数据没有这个问题，因为我们首先将所有文档连接在一起，然后将它们分割成等大小的部分。）

现在我们可以创建一个模型来对我们的文本进行分类：

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

在训练分类器之前的最后一步是从我们微调过的语言模型中加载编码器。我们使用`load_encoder`而不是`load`，因为我们只有编码器的预训练权重可用；默认情况下，如果加载不完整的模型，`load`会引发异常：

In [ ]:
learn = learn.load_encoder('finetuned')

### 微调分类器

最后一步是使用有区别的学习率和*逐步解冻*进行训练。在计算机视觉中，我们通常会一次性解冻整个模型，但对于NLP分类器，我们发现逐层解冻可以真正产生差异：

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.347427,0.184480,0.929320,00:33


仅仅在一个时代，我们就得到了与<<chapter_intro>>中训练相同的结果：还不错！我们可以传递`-2`给`freeze_to`来解冻除了最后两个参数组之外的所有层：

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.247763,0.171683,0.934640,00:37


然后我们可以解冻更多一些，并继续训练：

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.193377,0.156696,0.941200,00:45


最后，整个模型！

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.172888,0.153770,0.943120,01:01
1,0.161492,0.155567,0.942640,00:57


我们达到了94.3%的准确率，这在三年前是最先进的性能。通过在所有逆序阅读的文本上训练另一个模型，并平均这两个模型的预测，我们甚至可以达到95.1%的准确率，这是ULMFiT论文引入的最先进技术。它只是在几个月前被打破，通过微调一个更大的模型并使用昂贵的数据增强技术（将句子翻译成另一种语言然后再翻译回来，使用另一个模型进行翻译）。

使用预训练模型让我们构建了一个相当强大的微调语言模型，无论是生成假评论还是帮助对它们进行分类。这是令人兴奋的事情，但也要记住这项技术也可能被用于不良目的。

## 虚假信息和语言模型

在深度学习语言模型广泛可用之前，基于规则的简单算法就可以被用来创建欺诈性账户并试图影响政策制定者。Jeff Kao，现在是一名在ProPublica的计算记者，分析了针对美国联邦通信委员会（FCC）2017年提议废除网络中立性的评论。在他的文章["More than a Million Pro-Repeal Net Neutrality Comments Were Likely Faked"](https://hackernoon.com/more-than-a-million-pro-repeal-net-neutrality-comments-were-likely-faked-e9f0e3ed36a6)中，他报告了如何发现一大群反对网络中立性的评论，这些评论似乎是通过某种填表式的邮件合并生成的。在<<disinformation>>中，Kao通过颜色编码假评论，以突出它们的公式化特性。

<img src="images/ethics/image16.png" width="700" id="disinformation" caption="Comments received by the FCC during the net neutrality debate">

根据Jeff Kao的估计，“在2200万条以上的评论中...不到80万条可以被认为是真正独特的”，并且“超过99%的真正独特的评论支持保持网络中立性”。

考虑到自2017年以来语言建模方面的进步，现在这样的欺诈性活动几乎不可能被抓住。你现在可以利用所有必要的工具来创建一个有说服力的语言模型——也就是说，能够生成上下文适当、可信的文本的东西。它不一定完全准确或正确，但它将是可信的。想想这项技术在与我们近年来了解到的虚假信息活动结合时意味着什么。看看在<<ethics_reddit>>中展示的Reddit对话，其中一个基于OpenAI的GPT-2算法的语言模型正在与自己讨论美国政府是否应该削减国防开支。

<img src="images/ethics/image14.png" id="ethics_reddit" caption="An algorithm talking to itself on Reddit" alt="An algorithm talking to itself on Reddit" width="600">

在这种情况下，虽然明确说明了使用了算法，但想象一下，如果一个恶意行为者决定在社交网络上发布这样的算法会发生什么。他们可以慢慢地、谨慎地进行，允许算法随着时间的推移逐渐发展追随者和信任。拥有数百万账户执行此操作并不需要太多资源。在这种情况下，我们可以很容易地想象达到一个点，即在线上的绝大多数话语都来自机器人，而没有人会意识到这种情况正在发生。

我们已经开始看到机器学习被用来生成身份的例子。例如，<<katie_jones>>展示了一个名为Katie Jones的LinkedIn个人资料。

<img src="images/ethics/image15.jpeg" width="400" id="katie_jones" caption="Katie Jones's LinkedIn profile">

Katie Jones在LinkedIn上与华盛顿主流智库的几位成员建立了联系。但她并不存在。你所看到的图片是由生成对抗网络自动生成的，实际上并没有一个名叫Katie Jones的人从战略与国际研究中心（Center for Strategic and International Studies）毕业。

许多人认为或希望算法能在这里为我们提供保护——我们将开发出能够自动识别自动生成内容的分类算法。然而，问题在于，这将始终是一场军备竞赛，在这场竞赛中，更好的分类（或鉴别器）算法可以用来创建更好的生成算法。

## 总结

在本章中，我们探讨了fastai库开箱即用的最后一类应用：文本。我们看到了两种类型的模型：可以生成文本的语言模型，以及确定评论是正面还是负面的分类器。为了构建一个最先进的分类器，我们使用了一个预训练的语言模型，将其微调到我们任务的语料库上，然后使用它的主体（编码器）加上一个新的头部来进行分类。

在我们结束这一部分之前，我们将看看fastai库如何帮助你为特定问题组装数据。

## 问卷调查

1. **自我监督学习（Self-supervised learning）**：是一种机器学习方法，其中模型通过从数据本身生成监督信号来学习任务。在自然语言处理中，这通常涉及到预测文本中的下一个单词或填充缺失的单词，而不需要显式的标签。
2. **语言模型（Language model）**：是一种统计模型，用于预测文本序列中下一个单词的概率。它可以帮助我们理解语言的结构和单词之间的关系。
3. **为什么语言模型被认为是自我监督的？** 因为语言模型通常通过预测文本中的下一个单词来训练，这个过程不需要外部的标签，而是利用文本本身的结构作为监督信号。
4. **自我监督模型通常用于什么？** 它们通常用于理解语言的统计特性，如单词的共现概率，以及在自然语言处理任务中作为预训练模型。
5. **为什么我们要微调语言模型？** 微调是为了使预训练的语言模型适应特定的任务，如情感分析或文本分类，通过在特定数据集上进一步训练来提高模型在该任务上的表现。
6. **创建最先进的文本分类器的三个步骤是什么？** 首先，使用预训练的语言模型；其次，将其微调到特定任务的语料库上；最后，使用微调后的模型的编码器部分，加上一个新的分类头部来进行分类。
7. **50,000个未标记的电影评论如何帮助我们为IMDb数据集创建更好的文本分类器？** 这些评论可以用于训练一个通用的语言模型，然后这个模型可以被微调到IMDb的特定任务上，从而提高分类器的性能。
8. **为语言模型准备数据的三个步骤是什么？** 首先，对文本进行预处理，如清洗和标准化；其次，进行令牌化，将文本分割成可管理的单元；最后，进行数值化，将令牌转换为模型可以理解的数字。
9. **什么是令牌化（Tokenization）？为什么我们需要它？** 令牌化是将文本分割成单词、字符或其他有意义的单元的过程。我们需要它因为神经网络无法直接处理原始文本，它们需要将文本转换为数值形式。
10. **命名三种不同的令牌化方法。** 基于字符的令牌化、基于单词的令牌化、基于子词的令牌化（如Byte Pair Encoding或WordPiece）。
11. **什么是`xxbos`？** `xxbos`是一个特殊的令牌，表示文本序列的开始。在语言模型中，它帮助模型识别序列的起点。
12. **fastai在令牌化过程中应用的四条规则是什么？** 这些规则可能包括：移除特殊字符、转换为小写、替换重复字符等。
13. **为什么重复的字符会被替换为显示重复次数和重复字符的令牌？** 这样做是为了减少模型的复杂性，避免处理大量的重复字符，同时保留文本的结构信息。
14. **什么是数值化（Numericalization）？** 数值化是将令牌转换为数字索引的过程，以便神经网络可以处理。
15. **为什么有些单词可能被替换为“未知词”令牌？** 这可能是因为词汇表中没有这些单词，或者它们在训练数据中很少出现。
16. **如果批次大小为64，张量的第一行代表数据集中的前64个令牌。那么这个张量的第二行包含什么？第二批次的第一行包含什么？（注意：学生经常在这个问题上出错！确保在书的网站上检查你的答案。）** 第二行包含接下来的64个令牌，第二批次的第一行包含该批次的起始令牌。
17. **为什么我们需要在文本分类中进行填充？为什么在语言建模中不需要？** 在文本分类中，我们需要保持批次中所有文本的长度一致，以便输入到神经网络。而在语言建模中，模型是预测下一个单词，所以不需要保持固定长度。
18. **NLP的嵌入矩阵包含什么？它的形状是什么？** 嵌入矩阵包含词汇表中每个单词的向量表示。它的形状通常是词汇表大小（行数）乘以嵌入维度（列数）。
19. **什么是困惑度（Perplexity）？** 困惑度是衡量语言模型预测文本序列的能力的指标，它是模型预测概率的指数平均值的倒数。
20. **为什么我们必须将语言模型的词汇表传递给分类器数据块？** 这样做是为了确保在分类任务中使用的嵌入与在语言模型中学习到的嵌入一致。
21. **什么是逐步解冻（Gradual unfreezing）？** 逐步解冻是在微调过程中逐步解冻模型的层，以便在训练过程中逐步引入更多的模型参数。
22. **为什么文本生成总是可能领先于自动识别机器生成的文本？** 因为生成文本的模型（如GPT-3）在创建逼真文本方面非常强大，而检测这些文本的模型（如用于区分真实和生成文本的分类器）可能需要更多的时间和数据来达到相同的性能水平。

### 进一步研究

1. 了解关于语言模型和虚假信息的知识。当今最好的语言模型是什么？看看它们的一些输出。你觉得它们有说服力吗？一个恶意行为者如何最好地利用这样的模型来制造冲突和不确定性？

1. 鉴于模型不太可能始终如一地识别机器生成的文本的限制，处理利用深度学习的大规模虚假信息活动可能需要哪些其他方法？